### GÖKSU GÜZ 150180715

In [1]:
import numpy as np
import os
import cv2
import moviepy.editor as mpy
from matplotlib import pyplot as plt

In [2]:
background = cv2.imread('Malibu.jpg')
cv2.imshow('Background Image Window', background)
print(background.shape)

(776, 1998, 3)


In [3]:
background_height = background.shape[0]
background_width = background.shape[1]
ratio = 360/background_height
background = cv2.resize(background,(int(background_width*ratio), 360))
print(background.shape)

(360, 926, 3)


In [4]:
directory = 'cat/'
R = []
G = []
B = []
images_list = []
ref = cv2.imread('target.jpg')

for number in range(180):
    
    filename = directory + 'cat_' + str(number) + '.png'
    image = cv2.imread(filename)
    image2 = cv2.flip(image,1)
    
    arr1 = image2[:,:,0]
    arr2 = image2[:,:,1]
    arr3 = image2[:,:,2]
    
    histr1, bin_edges = np.histogram(arr1, density=True, bins = 256)
    histr2, bin_edges = np.histogram(arr2, density=True, bins = 256)
    histr3, bin_edges = np.histogram(arr3, density=True, bins = 256)
    
    R.append(histr1)
    G.append(histr2)
    B.append(histr3)
    
R = np.array(R)
G = np.array(G)
B = np.array(B)

columns_averageR = R.mean(axis=0)
columns_averageG = G.mean(axis=0)
columns_averageB = B.mean(axis=0)

#for Reference Image
arrRef1= ref[:,:,0]
arrRef2= ref[:,:,1]
arrRef3= ref[:,:,2]

histRef1, bin_edges = np.histogram(arrRef1, bins = 256)
histRef2, bin_edges = np.histogram(arrRef2, bins = 256)
histRef3, bin_edges = np.histogram(arrRef3, bins = 256)

#For Soruce Image 
#R
cdf1 = np.cumsum(columns_averageR/sum(columns_averageR))
#G
cdf2 = np.cumsum(columns_averageG/sum(columns_averageG))
#B
cdf3 = np.cumsum(columns_averageB/sum(columns_averageB))

#For Reference Image
#R
cdfRef1 = np.cumsum(histRef1/sum(histRef1))
#G
cdfRef2 = np.cumsum(histRef2/sum(histRef2))
#B
cdfRef3 = np.cumsum(histRef3/sum(histRef3))

In [5]:
LUTR = np.zeros((256, 1))
LUTG = np.zeros((256, 1))
LUTB = np.zeros((256, 1))

gj = 0
for gi in range(256):
    while cdfRef1[gj]<cdf1[gi] and gj<255:
        gj = gj+1
    LUTR[gi] = gj
gj = 0
for gi in range(256):
    while cdfRef2[gj]<cdf2[gi] and gj<255:
        gj = gj+1
    LUTG[gi] = gj
    
gj = 0
for gi in range(256):
    while cdfRef3[gj]<cdf3[gi] and gj<255:
        gj = gj+1
    LUTB[gi] = gj

In [6]:
def applyLut(img):
    x = np.shape(img)
    x = np.array(x)
    x = x[0]
    for i in range(x):
        img[i,0] = LUTR[img[i,0]]
        img[i,1] = LUTG[img[i,1]]
        img[i,2] = LUTB[img[i,2]] 
    return img
    

In [7]:
for number in range(180):
    filename = directory + 'cat_' + str(number) + '.png'
    image = cv2.imread(filename)
    image2 = cv2.flip(image,1)
   
    foreground = np.logical_or(image[:,:,1] < 180, image[:,:,0] > 150)
    foreground2 = np.logical_or(image2[:,:,1] < 180, image2[:,:,0] > 150)

    nonzero_x, nonzero_y = np.nonzero(foreground)
    nonzero_cat_values = image[nonzero_x, nonzero_y, :]
    
    nonzero_x2, nonzero_y2 = np.nonzero(foreground2)
    nonzero_cat_values2 = image2[nonzero_x2 , nonzero_y2, :]
    nonzero_cat_values2 = applyLut(nonzero_cat_values2)
    
    new_frame = background.copy()
    new_frame[nonzero_x, nonzero_y, :] = nonzero_cat_values
    new_frame[nonzero_x2, nonzero_y2+286, :] = nonzero_cat_values2 
    new_frame = new_frame[:,:,[2,1,0]]
    images_list.append(new_frame)

In [10]:
print(np.array(images_list).shape)
clip = mpy.ImageSequenceClip(images_list, fps = 25)
audio = mpy.AudioFileClip('selfcontrol_part.wav').set_duration(clip.duration)
clip = clip.set_audio(audioclip = audio)
clip.write_videofile('part2_video.mp4', codec = 'libx264')

chunk:   0%|                                                                         | 0/159 [00:00<?, ?it/s, now=None]

(180, 360, 926, 3)
Moviepy - Building video part2_video.mp4.
MoviePy - Writing audio in part2_videoTEMP_MPY_wvf_snd.mp3


t:  17%|███████████▍                                                       | 31/181 [00:00<00:00, 161.83it/s, now=None]

MoviePy - Done.
Moviepy - Writing video part2_video.mp4



Moviepy - Done !
Moviepy - video ready part2_video.mp4
